In [19]:
import os
import keras
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools
from sklearn import metrics

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
import matplotlib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

In [24]:
path = "./data/"

In [2]:
train = pd.read_csv('data/Train_data/dataset_rev4_train.csv')
#remove object
train = train.select_dtypes(exclude=['object'])
#replace misssing value
train.fillna(0,inplace=True)

In [3]:
test = pd.read_csv('data/test_data/dataset_rev4_test.csv')
#buffer datetime
buffer_datetime_test = test.datetime
#remove object
test = test.select_dtypes(exclude=['object'])
#replace misssing value
test.fillna(0,inplace=True)

In [4]:
print('dimension of train:', train.shape)
print('dimension of test:', test.shape)

dimension of train: (131586, 6)
dimension of test: (16147, 5)


In [5]:
print("features:",list(train.columns))

features: ['I', 'T', 'UV', 'WS', 'RH', 'P']


In [6]:
from sklearn.ensemble import IsolationForest
clf = IsolationForest(max_samples = 100, random_state = 42)
clf.fit(train)
y_noano = clf.predict(train)
y_noano = pd.DataFrame(y_noano, columns = ['Top'])
y_noano[y_noano['Top'] == 1].index.values

train = train.iloc[y_noano[y_noano['Top'] == 1].index.values]
train.reset_index(drop = True, inplace = True)
print("Number of Outliers:", y_noano[y_noano['Top'] == -1].shape[0])
print("Number of rows without outliers:", train.shape[0])

Number of Outliers: 13159
Number of rows without outliers: 118427


In [7]:
import warnings
warnings.filterwarnings('ignore')

col_train = list(train.columns)
col_train_bis = list(train.columns)

col_train_bis.remove('P')

mat_train = np.matrix(train)
mat_test  = np.matrix(test)

mat_new = np.matrix(train.drop('P',axis = 1))
mat_y = np.array(train.P).reshape((118427,1))

prepro_y = MinMaxScaler()
prepro_y.fit(mat_y)

prepro = MinMaxScaler()
prepro.fit(mat_train)

prepro_test = MinMaxScaler()
prepro_test.fit(mat_new)

train = pd.DataFrame(prepro.transform(mat_train),columns = col_train)
test  = pd.DataFrame(prepro_test.transform(mat_test),columns = col_train_bis)

In [8]:
# List of features
COLUMNS = col_train #column train (x train)
FEATURES = col_train_bis  #column train-label (x test)
LABEL = "P"

# Columns
feature_cols = FEATURES #(x test)

# Training set and Prediction set with the features to predict
training_set = train[COLUMNS] #column train (x train)
prediction_set = train.P # column P

In [9]:
print(type(training_set))
print(type(prediction_set))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [10]:
x_train, x_test, y_train, y_test = train_test_split(training_set[FEATURES] , prediction_set, test_size=0.33, random_state=42)

In [11]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(79346, 5)
(39081, 5)
(79346,)
(39081,)


In [12]:
y_train = pd.DataFrame(y_train, columns = [LABEL])
training_set = pd.DataFrame(x_train, columns = FEATURES).merge(y_train, left_index = True, right_index = True)
training_sub = training_set[col_train]

In [13]:
y_test = pd.DataFrame(y_test, columns = [LABEL])
testing_set = pd.DataFrame(x_test, columns = FEATURES).merge(y_test, left_index = True, right_index = True)

In [14]:
seed = 7
np.random.seed(seed)

# Model
model = Sequential()
model.add(Dense(200, input_dim=5, kernel_initializer='normal', activation='relu'))
model.add(Dense(100, kernel_initializer='normal', activation='relu'))
model.add(Dense(50, kernel_initializer='normal', activation='relu'))
model.add(Dense(25, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))

In [15]:
# Compile model
model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adadelta())

In [16]:
feature_cols = training_set[FEATURES]
labels = training_set[LABEL].values

In [17]:
model.fit(np.array(feature_cols), np.array(labels), epochs=100, batch_size=10)

Epoch 1/100
79346/79346 [==============================] - 14s 175us/step - loss: 0.0047
Epoch 2/100
79346/79346 [==============================] - 12s 147us/step - loss: 0.0039
Epoch 3/100
79346/79346 [==============================] - 11s 133us/step - loss: 0.0038
Epoch 4/100
79346/79346 [==============================] - 12s 145us/step - loss: 0.0038
Epoch 5/100
79346/79346 [==============================] - 11s 133us/step - loss: 0.0037
Epoch 6/100
79346/79346 [==============================] - 13s 159us/step - loss: 0.0037
Epoch 7/100
79346/79346 [==============================] - 12s 155us/step - loss: 0.0037
Epoch 8/100
79346/79346 [==============================] - 11s 142us/step - loss: 0.0036
Epoch 9/100
79346/79346 [==============================] - 11s 135us/step - loss: 0.0036
Epoch 10/100
79346/79346 [==============================] - 11s 140us/step - loss: 0.0036
Epoch 11/100
79346/79346 [==============================] - 12s 150us/step - loss: 0.0036
Epoch 12/100
79346/

In [20]:
# Predict
pred = model.predict(x_test)

# Measure MSE error.  
score = metrics.mean_squared_error(pred,y_test)
print("Final score (MSE): {}".format(score))

Final score (MSE): 0.003421202439713599


In [21]:
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

Final score (RMSE): 0.05849104580800038


In [25]:
# save entire network to HDF5 (save everything, suggested)
model.save(os.path.join(path,"R13_model.h5"))

In [26]:
# save neural network structure to JSON (no weights)
model_json = model.to_json()
with open(os.path.join(path,"R13_model.json"), "w") as json_file:
    json_file.write(model_json)

In [27]:
# save neural network structure to YAML (no weights)
model_yaml = model.to_yaml()
with open(os.path.join(path,"R13_model.yaml"), "w") as yaml_file:
    yaml_file.write(model_yaml)